In [ ]:
# imports
from contextlib import closing
import sqlite3
import re
from dateutil import parser as date_parser

In [2]:
# testing

db_path = "../../data/vatican_texts.db"

In [5]:

def load_popes_only(db_path):
    """
    Returns all popes from the provided database
    """
    with closing(sqlite3.connect(database=db_path)) as connection:
        with connection:
            connection.row_factory = sqlite3.Row
            
            query_results = connection.execute("SELECT pope_name, pope_number, place_of_birth, _pope_id FROM popes")

            popes = [{
                'pope_name': row["pope_name"],
                'pope_number': row["pope_number"],
                'place_of_birth': row['place_of_birth'],
                'pope_id': row['_pope_id']
            } for row in query_results]
            # popes = [
            #     {
            #         'pope_name': row['pope_name'],
            #         'pope_data': {k:row[k] for k in row.keys() if k != 'pope_name'}
            #     }
            # for row in rows]
    return popes

popes = load_popes_only(db_path)

In [6]:
popes

[{'pope_name': 'Benedict XVI',
  'pope_number': '265th Pope of the Catholic Church',
  'place_of_birth': 'Marktl, Germany',
  'pope_id': 1},
 {'pope_name': 'John Paul II',
  'pope_number': '264th Pope of the Catholic Church',
  'place_of_birth': 'Wadowice, Poland',
  'pope_id': 2},
 {'pope_name': 'Paul VI',
  'pope_number': '262nd Pope of the Catholic Church',
  'place_of_birth': 'Concesio, Italy',
  'pope_id': 3},
 {'pope_name': 'Francis',
  'pope_number': '266th Pope of the Catholic Church',
  'place_of_birth': 'Buenos Aires, Argentina',
  'pope_id': 4},
 {'pope_name': 'Leo XIV',
  'pope_number': '267th Pope of the Catholic Church',
  'place_of_birth': 'Chicago, Illinois',
  'pope_id': 5}]

In [7]:
def load_speeches_only(db_path, popes):
    pope_data = {}
    with closing(sqlite3.connect(db_path)) as connection:
        with connection:
            connection.row_factory = sqlite3.Row
            for pontiff in popes:
                query_results = connection.execute(
                    """
                    SELECT title, date, section
                    FROM texts
                    WHERE pope_id = ?
                    ORDER by date
                    """,
                    (pontiff['pope_id'],)
                )
                pontiff_text_data = [
                    {
                        'title': qr['title'],
                        'date': qr['date'],
                        'category': qr['section']
                    }
                    for qr in query_results
                ]
                pope_data[pontiff['pope_name']] = pontiff_text_data
    return pope_data

load_speeches_only(db_path, popes)

{'Benedict XVI': [{'title': 'To the members of the Vatican City State Police Force (December 31, 2005)',
   'date': None,
   'category': 'speeches'},
  {'title': 'Visit to the Pontifical Dispensary of "Santa Marta" (December 30, 2005)',
   'date': None,
   'category': 'speeches'},
  {'title': 'To the participants of the International Congress of the Pueri Cantores (December 30, 2005)',
   'date': None,
   'category': 'speeches'},
  {'title': 'To the employees who renovated the Private Apartments in the Vatican Apostolic Palace (December 23, 2005)',
   'date': None,
   'category': 'speeches'},
  {'title': 'To the Ambassador of the United Kingdom of Great Britain and Northern Ireland to the Holy See (December 23, 2005)',
   'date': None,
   'category': 'speeches'},
  {'title': 'Christmas greetings to the Members of the Roman Curia and Prelature (December 22, 2005)',
   'date': None,
   'category': 'speeches'},
  {'title': 'To the singers of the Pontifical Musical Chorus of the "Sistine" 

In [8]:
# Entries without dates often have it in the title, as such: Title So and So (DATE). So this helps us extract it from.

def extract_date_from_title(sentence:str):
    """
    obtains the date from the end of a sentence.
    Supports: (Month DD, YYYY) OR (DD Month YYYY) OR (DD[nd, rd, st] Month YYYY)
    """
    # pattern = r"\((?:.*,\s+)?([A-Z][a-z]+ \d{1,2}, \d{4}|\d{1,2}(?:st|nd|rd|th)? [A-Z][a-z]+ \d{4})\)$"
    pattern = r"""
        \(                # Opening parenthesis
        (?:.*,\s+)?       # Optional location and comma
        (                 # Start capture group for date
            [A-Z][a-z]+\s+\d{1,2},\s+\d{4}     # Month DD, YYYY
            |                                  # OR
            \d{1,2}(?:st|nd|rd|th)?\s+[A-Z][a-z]+\s+\d{4} # DD Month YYYY
        )                 # End capture group
        \)                # Closing parenthesis
        $                 # End of line
    """
    match = re.search(pattern, sentence.strip(), re.IGNORECASE | re.VERBOSE)
    if match:
        return match.group(1)  # Returns the content inside the parentheses
    return None

# test
t1 = extract_date_from_title('To the members of the Vatican City State Police Force (December 31, 2005)')
t2 = extract_date_from_title('(10 december 2025)')
t3 = extract_date_from_title('(1st June 2025)')
t4 = extract_date_from_title('(Castel Gandolfo, 1st October 2025)')
print(t1,t2,t3,t4)

December 31, 2005 10 december 2025 1st June 2025 1st October 2025


In [9]:
def clean_papacy_date(date_old_format:str):
    """
    Converting from the following format: DD,HH.MMM.YYYY, but months are in roman
    """
    roman_map = {
        'I':'01', 'II':'02', 'III':'03', 'IV':'04', 'V':'05', 'VI':'06',
        'VII':'07', 'VIII':'08', 'IX':'09', 'X':'10', 'XI':'11', 'XII':'12'
    }
    temp = date_old_format.split(',')
    day = temp[0]
    if len(day) == 1:
        day = '0' + day
    month, year = temp[1].split('.')[1:]
    month = roman_map[month]
    new_date = f"{year}-{month}-{day}"
    return new_date

In [50]:
def clean_text_date(date:str):
    """
    Turns date from a given format to YYYY-MM-DD
    Supports: (Month DD, YYYY) OR (DD Month YYYY) OR (DD[nd, rd, st] Month YYYY)
    """
    # STEP 0 - date is non
    if date is None:
        return date
    # STEP 1 - Try parser
    try:
        new_date = date_parser.parse(date, fuzzy=True)
        return new_date.strftime("%Y-%m-%d")
    except (date_parser.ParserError) as e:
        # print(f"Parser didn't help, trying manually. Error: {e}")
        pass
    
    # STEP 2 - try manually
    parts = date.split(' ')
    discard_symbols = [',', 'st', 'nd', 'rd', 'th']
    for symbol in discard_symbols:
        parts = [p.replace(symbol, '') for p in parts]
    year = parts[2]
    try:
        day = int(parts[0])
        month = parts[1]
    except ValueError:
        month = parts[0]
        day = int(parts[1])
    day = str(day)
    if len(day) == 1:
        day = '0' + day
    
    # clean month
    month_map = {
        "gennaio": "01", "febbraio": "02", "marzo": "03", "aprile": "04",
        "maggio": "05", "giugno": "06", "luglio": "07", "agosto": "08",
        "settembre": "09", "ottobre": "10", "novembre": "11", "dicembre": "12"
    }
    try:
        month = int(month)
    except ValueError:
        # convert from word to number
        month = month_map.get(month, month)

    new_date = f"{year}-{month}-{day}"
    return new_date

with open('test_dates.txt', 'r') as f:
    test_dates = f.readlines()

for dt in test_dates:
    print(clean_text_date(dt))

2005-12-31
2025-12-10
2025-06-01
2025-10-01
1985-04-01
2018-04-01


In [55]:
def load_speeches_join_pope(db_path):
    """
    popes_data is a dict that looks like this:
    {
        "<pope_name>": {"pope_name": str,
                        "papacy_began": str,
                        "texts": [{ "title": str,
                                    "date": str | date,
                                    "category": str}, ...]
        },...}
    """
    with closing(sqlite3.connect(db_path)) as connection:
        with connection:
            connection.row_factory = sqlite3.Row
            query = (
                """
                SELECT p.pope_name, t.title, t.date, t.section, p.pontificate_begin
                FROM popes p
                JOIN texts t ON p._pope_id = t.pope_id
                ORDER BY p.pontificate_begin, t.date
                """
            )

            query_results =  connection.execute(query)
            popes_data = {}

            for row in query_results:
                pope_name = row['pope_name']
                if pope_name not in popes_data:
                    popes_data[pope_name] = {
                        'pope_name': pope_name,
                        'papacy_began': clean_papacy_date(row['pontificate_begin']),
                        'texts': []
                    }
                # reformat dates
                if row['date'] is not None:
                    new_date = clean_text_date(row['date'])
                else:
                    new_date = extract_date_from_title(row['title'])
                    new_date = clean_text_date(new_date)
                popes_data[pope_name]['texts'].append({
                    'title': row["title"],
                    'date': new_date,
                    'category': row['section']
                })

    return popes_data

popes_data = load_speeches_join_pope(db_path)


In [57]:
def rearrange_pope_data(popes_data):
    for pdata in popes_data.values():

        pdata['texts'].sort(key=lambda x: (x['date'] is None, x['date']))
    return popes_data

popes_data = rearrange_pope_data(popes_data)

In [58]:
# This was me just visualizing the resulting dates better
with open('reformatted_dates.txt', 'w') as f:
    for pontiff, pontiff_data in popes_data.items():
        f.write(f"<<{pontiff}>>:\n\n")
        for text in pontiff_data['texts']:
            if text['date']:
                f.write(text['date'] + '\n')
            else:
                f.write('NONE FOUND\n')
        f.write('\n\n\n')

OK great, almost there, maybe one last thing would be to take all dates that are valid (get rid of nones, and other stragglers), and use those for our stats.
* Note: there are some not nones that are just randome other languages or possibly typos: jaunary(?), octobre (french), etc.

In [ ]:
def print_pope_data(popes_data):
    for _, pdata in popes_data.items():
        pope_stats = f"""{pdata['pope_name']}:\nNumber of texts: {len(pdata['texts'])}.\nPapacy began: {pdata['papacy_began']}\n"""
        print(pope_stats)
    return

print_pope_data(popes_data)

Francis:
Number of texts: 4619.
Papacy began: 2013-03-13

John Paul II:
Number of texts: 2719.
Papacy began: 1978-10-16

Benedict XVI:
Number of texts: 2932.
Papacy began: 2005-04-19

Paul VI:
Number of texts: 541.
Papacy began: 1963-06-21

Leo XIV:
Number of texts: 520.
Papacy began: 2025-05-08



In [ ]:
popes_data['Francis']['texts'][1]

{'title': 'Address to participants in the International Seminar on the Pope\'s proposal made in Evangelii Gaudium "Towards a more inclusive economy"',
 'date': None,
 'category': 'speeches'}